In [46]:
import os
import json
import requests
from openai import OpenAI
import time

In [47]:
client: OpenAI = OpenAI()
api_key = os.environ["FMP_API_KEY"]

In [48]:
def get_income_statement(period:str ) -> str:
    url:str = f'https://financialmodelingprep.com/api/v3/income-statement/AAPL?period={period}&apikey={api_key}'
    response = requests.get(url)
    result = json.dumps(response.json())
    return result

def get_balance_sheet(period:str ) -> str:
    url:str = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/0000320193?period={period}&apikey={api_key}'
    response = requests.get(url)
    result = json.dumps(response.json())
    return result
def get_cash_flow_statement(period:str ) -> str:
    url:str = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/0000320193?period={period}&apikey={api_key}'
    response = requests.get(url)
    result = json.dumps(response.json())
    return result
def get_key_metrics(period:str ) -> str:
    url:str = f'https://financialmodelingprep.com/api/v3/key-metrics/AAPL?period={period}&apikey={api_key}'
    response = requests.get(url)
    result = json.dumps(response.json())
    return result
def get_financial_ratios(period:str ) -> str:
    url:str = f'https://financialmodelingprep.com/api/v3/ratios/AAPL?period={period}&apikey={api_key}'
    response = requests.get(url)
    result = json.dumps(response.json())
    return result
def get_financial_growth(period:str ) -> str:
    url:str = f'https://financialmodelingprep.com/api/v3/get_financial_growth/AAPL?period={period}&apikey={api_key}'
    response = requests.get(url)
    result = json.dumps(response.json())
    return result





In [49]:
tool = [
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "FMP's Income Statement API provides access to real-time income statement data for a wide range of companies, including public companies, private companies, and ETFs. This data can be used to track a company's profitability over time, to compare a company to its competitors, and to identify trends in a company's business.",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {
                        "type": "string",
                        "description": "period can have two values either annual or quarter.but at a time one value",
                    }
                },
                "required": ["period"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "The balance sheet is a financial statement that displays a company’s total assets, liabilities, and shareholder equity over a specific timeframe (quarterly or yearly). Investors can use this statement to determine if the company can fund its operations, meet its debt obligations, and pay a dividend.",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {
                        "type": "string",
                        "description": "period can have two values either annual or quarter.but at a time one value",
                    }
                },
                "required": ["period"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_cash_flow_statement",
            "description": "The cash flow statement is a financial statement that highlights how cash moves through the company, including both cash inflows and outflows. This statement shows the cash flows in 3 main categories “Operating Cash Flows”, “Investing Cash Flows”, and “Financing Cash Flows”, which help investors to understand if the company is making money or losing money by conducting business.",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {
                        "type": "string",
                        "description": "period can have two values either annual or quarter.but at a time one value",
                    }
                },
                "required": ["period"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_key_metrics",
            "description": "Get key financial metrics for a company, including revenue, net income, earnings per share (EPS), and price-to-earnings ratio (P/E ratio). Assess a company's financial performance and compare it to its competitors.",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {
                        "type": "string",
                        "description": "period can have two values either annual or quarter.but at a time one value",
                    }
                },
                "required": ["period"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_financial_ratios",
            "description": "Get financial ratios for a company, such as the P/B ratio and the ROE. Assess a company's financial health and compare it to its competitors.",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {
                        "type": "string",
                        "description": "period can have two values either annual or quarter.but at a time one value",
                    }
                },
                "required": ["period"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_financial_growth",
            "description": "",
            "parameters": {
                "type": "object",
                "properties": {
                    "period": {
                        "type": "string",
                        "description": "period can have two values either annual or quarter.but at a time one value",
                    }
                },
                "required": ["period"],
            },
        },
    },
]

In [50]:
available_functions = {
    "get_income_statement":get_income_statement,
    "get_cash_flow_statement":get_cash_flow_statement,
    "get_balance_sheet":get_balance_sheet,
    "get_key_metrics":get_key_metrics,
    "get_financial_ratios":get_financial_ratios,
    "get_financial_growth":get_financial_growth
}

In [ ]:
assistants = client.beta.assistants.create(
    name = "Financial Analyst Assistant",
    instructions = "Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends, etc, etc.",
    model = "gpt-3.5-turbo-1106",
    tools=tool
)


In [ ]:
thread = client.beta.threads.create()


In [53]:
messages = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "please give me the annual income statement report"
)

In [82]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistants.id
)

In [ ]:
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id= thread.id,
        run_id = run.id
    )
    

    if run.status == 'requires_action':
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        

        tool_output = []
        for tool_call in tool_calls:
            tool_id = tool_call.id
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                
                output = function_to_call(**function_args)
                
                tool_output.append(
                    {
                        "tool_call_id":tool_id,
                        "output":output
                    }
                )
        
        
        client.beta.threads.runs.submit_tool_outputs(
            thread_id = thread.id,
            run_id = run.id,
            tool_outputs = tool_output  
        )


        
    elif run.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id = thread.id,
            order = 'asc'
        )
        for message in messages:
            if message.role == 'user':
                print('\n\n')
                print('QUESTION:')
                print(f'{message.role} : {message.content[0].text.value}')
                print('============================================================================\n')
            else:
                print(f'{message.role}: {message.content[0].text.value}')
        
        break
    elif run.status == 'failed':
        print('Run Failed')
        break
    elif run.status in ['in_progress','queued']:
        print(f'Run is {run.status}. please waiting...')
        time.sleep(.5)
    else:
        print(f'unexpected status:{run.status} ')
        break

    